In [1]:
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.model.SeriesCT import SeriesCT

from library_dicom.dicom_processor.model.FusionPET_CT import FusionPET_CT

import SimpleITK as sitk 
import tensorflow as tf 
import numpy as np

In [2]:
input_shape = tuple(list((256,128,128)) + [2])
input_shape

(256, 128, 128, 2)

In [3]:
ct = '/home/deeplearning/AHL/AHL_Validated_DICOM/53011101871015/pet0/1.3.12.2.1107.5.1.4.41145.4.0.739399949082153/1.3.12.2.1107.5.1.4.41145.4.0.739399949082153'
pet = '/home/deeplearning/AHL/AHL_Validated_DICOM/53011101871015/pet0/1.3.12.2.1107.5.99.1.123456.2.0.15709579130482/1.3.12.2.1107.5.99.1.123456.2.0.15709579130482'

pet_nifti = '/media/deeplearning/Elements/AHL2011/AHL_NIFTI/53011101871015/PET0/1.2.250.1.38.2.1.102.140128091739.102948944_nifti_PT.nii'

In [4]:
series_pet_object = SeriesPT(pet)
series_pet_object.get_instances_ordered()

series_ct_object = SeriesCT(ct)

In [5]:
f = FusionPET_CT(series_pet_object, series_ct_object)

In [6]:
f.get_feature_pet_img()

((5.3067274, 5.3067274, -3.375),
 (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0),
 (-340.57017, -514.08843, -1849.001),
 [128, 128, 299])

In [7]:
f.get_feature_ct_img()

((0.9765625, 0.9765625, -2.4),
 (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0),
 (-249.51172, -424.51172, -1851.5),
 [512, 512, 421])

In [15]:
pet_img, ct_img = f.generate_pet_ct_img()

In [16]:
pet_img.GetSize()

(128, 128, 299)

In [36]:
extract = sitk.ExtractImageFilter()
extract.SetSize([128, 128, 0])
extract.SetIndex([0, 0, 0])
extracted_img = extract.Execute(pet_img)

In [9]:
f.calculate_new_origin(pet_img, mode='head')

(-256.93961640000003, -430.45787640000003, -3882.126)

In [10]:
concat, new_origin = f.resample(pet_img, ct_img, mode='head')

In [11]:
concat.shape

(256, 128, 128, 2)

In [12]:
cancat_img = f.save_nifti_fusion(pet_img, ct_img, '/home/test.nii', mode='head')

In [13]:
cancat_img.GetSize()

(128, 128, 256, 2)

In [42]:
modal_pp = dict(a_min=0.0, a_max=25.0, b_min=0.0, b_max=1.0, clip=True)

In [43]:
modal_pp['a_min']

0.0

In [14]:
pet_img = sitk.ReadImage(pet_nifti)
pet_img.GetSize()
#array = 299 128 128 sans transpose()

(128, 128, 299)